# Machine Learning Model for Dublin Bikes application

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import seaborn as sns
import tensorflow as tf
from sklearn.linear_model import LinearRegression

2023-04-04 10:30:07.292989: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df = pd.read_csv("dynamic_4_4_23.csv")
df.head() 

,number,name,bike_stands,available_bike_stands,available_bikes,status,s_date,s_time
0,42,SMITHFIELD NORTH,30,15,15,OPEN,2023-03-03,10:47:16
1,30,PARNELL SQUARE NORTH,20,19,1,OPEN,2023-03-03,10:45:35
2,54,CLONMEL STREET,33,21,12,OPEN,2023-03-03,10:49:41
3,108,AVONDALE ROAD,35,17,18,OPEN,2023-03-03,10:44:37
4,20,JAMES STREET EAST,30,12,17,OPEN,2023-03-03,10:44:23


In [4]:
df["availability"] = df["available_bikes"]/(df["available_bikes"] + df["available_bike_stands"])
df["availability"].round(2)

0          0.50
1          0.05
2          0.36
3          0.51
4          0.59
           ... 
1060225    0.40
1060226    0.00
1060227    0.00
1060228    0.53
1060229    0.37
Name: availability, Length: 1060230, dtype: float64

KeyError: 'date'

,number,name,bike_stands,available_bike_stands,available_bikes,status,s_date,s_time,availability
0,42,SMITHFIELD NORTH,30,15,15,OPEN,2023-03-03,10:47:16,0.500000
1,30,PARNELL SQUARE NORTH,20,19,1,OPEN,2023-03-03,10:45:35,0.050000
2,54,CLONMEL STREET,33,21,12,OPEN,2023-03-03,10:49:41,0.363636
3,108,AVONDALE ROAD,35,17,18,OPEN,2023-03-03,10:44:37,0.514286
4,20,JAMES STREET EAST,30,12,17,OPEN,2023-03-03,10:44:23,0.586207
